In [2]:
import pandas as pd
from itertools import combinations
from collections import Counter
import networkx as nx
import json
from tqdm import tqdm

c:\Users\aneas\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


[Get dataset here](https://nijianmo.github.io/amazon/index.html)

### Filtering

In [ ]:
df = pd.read_json('Movies_and_TV_5.json', lines = True)
df = df.loc[df['verified']]
df.drop(['reviewTime', 'image', 'vote', 'style', 'verified'], inplace=True, axis=1)


In [3]:
# df.head()

In [11]:
df.to_csv("tv.csv")

### Network

In [4]:
df = pd.read_csv('tv.csv')
df.drop(['Unnamed: 0'], inplace=True, axis=1)

In [ ]:
# grouped = df.groupby('asin')['reviewerID'].unique()
# for movie_id, reviewers in grouped.items():
#     if len(set(reviewers)) < 5:
#         print(f"Movie {movie_id} has only {len(set(reviewers))} unique reviewers.")
#         break

In [5]:
def filter_reviews(df):
    prev_shape = (0, 0)
    current_shape = df.shape

    while current_shape != prev_shape:
        prev_shape = current_shape

        # Filter movies with at least 5 unique reviewers
        movie_counts = df.groupby('asin')['reviewerID'].nunique()
        valid_movies = movie_counts[movie_counts >= 10].index
        df = df[df['asin'].isin(valid_movies)]

        # Filter reviewers who reviewed at least 5 unique movies
        reviewer_counts = df.groupby('reviewerID')['asin'].nunique()
        valid_reviewers = reviewer_counts[reviewer_counts >= 10].index
        df = df[df['reviewerID'].isin(valid_reviewers)]

        current_shape = df.shape

    return df

filtered_df = filter_reviews(df)

In [6]:
reviewer_counts = filtered_df.groupby('reviewerID')['asin'].nunique()
len(reviewer_counts[reviewer_counts > 500])

3

In [7]:
grouped = filtered_df.groupby('asin')['reviewerID'].unique()
len(grouped)

20244

In [8]:
sum(len(set(reviewers)) < 5 for reviewers in grouped.values)

0

In [9]:
edges = Counter()
for reviewers in tqdm(grouped.values, desc="Movies"):
    for pair in combinations(reviewers, 2):
        edges[pair] += 1

Movies: 100%|██████████| 20244/20244 [01:38<00:00, 206.30it/s]


In [10]:
edgelist = [(a, b, count) for (a, b), count in edges.items()]
len(edgelist)

96589998

In [11]:
thresholds = [5, 10, 20, 50, 100]
for t in thresholds:
    filtered = [(a, b, w) for (a, b, w) in edgelist if w > t]
    print(f"Edges with weight > {t}: {len(filtered)}")

Edges with weight > 5: 262524
Edges with weight > 10: 15907
Edges with weight > 20: 649
Edges with weight > 50: 1
Edges with weight > 100: 0


In [12]:
# df_edges = pd.DataFrame(
#     [(u, v, w) for (u, v), w in edges.items()],
#     columns=['reviewer1', 'reviewer2', 'weight']
# )
# df_edges.to_csv('edges.csv', index=False)

### Saving graph

In [13]:
def save_graph(graph_file, G):
    data = nx.readwrite.json_graph.node_link_data(G)
    with open(graph_file, "w") as f:
        json.dump(data, f)

def load_graph(graph_file):
    with open(graph_file, "r") as f:
        data = json.load(f)
    return nx.readwrite.json_graph.node_link_graph(data)